# 魔术方法



In [3]:
# __new__ 用途：
# __new__ 用在元编程当中，type构造类对象有关，metaclass
# __new__ 类中实例化使用，构造实例有关

class A:
    # __new__永远是静态方法
    def __new__(cls, *args, **kwargs):
        print('cls: ', cls)    # 类A
        print(args)
        print(kwargs)
        print('-' * 30)
        # 产生递归调用异常
        # 不能自己调自己 交给根基类object做
        # return cls(*args, **kwargs)

        # 为什么要传参cls: 根基类要对什么类型(cls)的类进行实例化
        # return object.__new__(cls)
        # return super(A, cls).__new__(cls)
        # instance = super().__new__(cls)  # super(A, cls) => object
        # instance.z = 200
        # cls.Z = 300    # 建议不要这么做 反复构造实例 Z属性会被反复覆盖
        # return instance
        # super().__new__ 是静态方法，所有参数都你自己给，没有绑定(自动注入)效果
        return super().__new__(cls)

    def __init__(self, x, y):
        self.xx = x
        self.yy = y


t = A(x=3, y=[4, 5])
print(type(t), t)    # None
# 1. 实例化  __new__(cls, *args, **kwargs) => 制造实例 (从无到有)
# 2. 初始化  __init__(self)
print(t.__dict__)
# print(t.__class__.__dict__)

# __init__()方法不能有返回值，也就是只能return None
# A(x=3, y=[4, 5])
# 1. obj = object.__new__(A) => obj = instance
# 2. obj.__init__(3, [4, 5])
# 3. return obj

# 魔术方法，一般使用要小心，弄不好就会递归

cls:  <class '__main__.A'>
()
{'x': 3, 'y': [4, 5]}
------------------------------
<class '__main__.A'> <__main__.A object at 0x0000019C4FE89A48>
{'xx': 3, 'yy': [4, 5]}


In [ ]:
* 可视化

In [10]:
from pathlib import Path


p1 = Path('/etc/sysconfig/network')    # p1是Path实例
print(p1)      # 直接作用： Path.__str__
print([p1])    # 间接作用： Path.__repr__
# Path类的 __str__ 和 __repr__ 不一样
print(bytes(p1))

# 小心，可能报错 p1是Path类的实例
# 这里不报错因为：Python内建函数很多都已经能够使用Path实例
# with open(str(p1) as f: 
# with open(p1) as f: 
#     pass
# 如果换第三方函数（文件对象 or 路径字符串）则会报错
# 第三方函数(str(p1)) p1.__str__ 返回路径字符串
# 第三方函数(str(p1)) p1.__repr__  => WindowsPath实例
# 第三方函数(p1) p1.__repr__  => WindowsPath实例

\etc\sysconfig\network
[WindowsPath('/etc/sysconfig/network')]
b'\\etc\\sysconfig\\network'


In [8]:
print(p1.__repr__()), print(p1.__str__())

WindowsPath('/etc/sysconfig/network')
\etc\sysconfig\network


(None, None)

In [ ]:
# __new__ 用途：
# __new__ 用在元编程当中，type构造类对象有关，metaclass
# __new__ 类中实例化使用，构造实例有关

class A:  # mro object.__str__兜底(没有则继承object)
    # 魔术方法，一般使用要小心，弄不好就会递归
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):    # 字符串表达
        return "<{} {}, {}>, addr={}".format(self.__class__.__name__, self.x, self.y, hex(id(self)))   # 只是一种表达

    def __repr__(self):
        return "repr"    # str

    # __str__ = __repr__  # 如果两个函数一样 定义一个即可

    def __bytes__(self):  # 解决bytes(t)
        return str(self).encode()

t = A(x=3, y=[4, 5])
print(bytes(t))  # bytes没人兜底
print(t)  # bytes没人兜底
# print(type(t), t, '+++++')
# print\str\format 三个函数直接作用对象上，那么就会调用 对象.__str__ 魔术方法
# 如果不是 调用对象.__repr__ (间接作用调用)
# repr(对象) => 直接调用 对象.__repr__  => object.__repr__
# 如果 __str__ 魔术方法没有定义 尝试调用__repr__方法；如果没有__repr__，就找基类的__repr__
# 根基类 只定义 object.__repr__就够了
# print(str(t))
# print(t)
# print("{}".format(t))
# print('-' * 30)
#
# print((t,))
# print([t])
# print(str([t]))
# print({t})


# <__main__.A object at 0x000002107FF94948>  通用实例的表达
# 没有写__str__魔术方法 则去继承根基类object的


# class B(list):
#     pass
#
#
# # print(B())    # 相当于 list() 符号性的表达，给人看的
# # print(list())
# b1 = B()
# b1.append(5)
# b1.extend(range(5))
# print(b1)

* print()/str()/format() 
  - 这三个函数**直接作用**到对象上，那么就会调用 对像.__str__ 魔术方法
  
* print([obj])/print((obj,))...
  - 如果不是这三个函数直接作用 而是**间接做用** 会调用 对象.__repr__ 魔术方法


  * __str__： str()函数、format()函数、print()函数调用，需要返回对象的字符串表达。如果没有定义，就去调用__repr__方法返回字符串表达，如果__repr__没有定义，就直接返回对象的内存地址信息: hex(id(self))
  
  * __repr__: 内建函数repr()对一个对象获取**字符串**表达。调用__repr__方法返回字符串表达，如果__repr__也没有定义，就直接返回object的定义，就是显示内存地址信息
  
  * __bytes__: bytes()函数调用，返回一个对象的bytes表达，即返回bytes对象

In [11]:
class A:
    def __init__(self, name, age=18):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return 'repr: {}, {}'.format(self.name, self.age)
    
    def __str__(self):
        return 'str: {}, {}'.format(self.name, self.age)
    
    def __bytes__(self):
        import json
        return json.dumps(self.__dict__).encode()

In [14]:
tom = A('TOM')

In [15]:
print(tom)

str: TOM, 18


In [17]:
print('{}'.format(tom))

str: TOM, 18


In [19]:
print([tom])

[repr: TOM, 18]


In [20]:
print([str(tom)])

['str: TOM, 18']


In [21]:
print(bytes(tom))

b'{"name": "TOM", "age": 18}'


In [22]:
bool(None)

False

* bool

* 等效False  仅仅是Python的规定 Python的空容器是等效Fasle，其他语言未必如此
  - 空容器：[] () {} set()
  - 0 '' b'' None
  
  __bool__: 对对象使用内建函数bool()，或者对象放在逻辑表达式的位置，调用这个函数返回布尔值。没有定义__bool__()，就找__len__() 返回长度，非0为真。如果__len__()也没有定义，那么所有实例都返回真

In [ ]:
# class A:
#     pass
#
#
# if A:  # 类对象 默认为真
#     print('real A')
#
# if A():  # 默认类的实例为真 == if bool(A())
#     print('read A()')


class B:
    # def __bool__(self):
    #     return True #  True or False or bool('abc')
    #     # return bool(self)    # 产生递归错误

    def __len__(self):
        print('len +++')
        return 0  # 0 False; >0 True 非空


# print(bool(B))
t = B()
print(bool(t))  # B().__bool__()

In [ ]:
* 比较运算符重载

== 是**同类型**比较内容
**同类型**不知道怎么比较，转而比较内存地址是否相同（is）
不同类型无法比较，直接返回False

  * __eq__ 等于可以推断不等于
  * __gt__ 大于可以推断小于
  * __ge__ 大于等于可以推断小于等于
也就是用3个方法，就可以把所有比较解决掉


In [ ]:
# from pathlib import Path
#
#
# p1 = Path('/etc/syscofig/network')
# p2 = Path('a')
# p1 / 'abc'
# 'abc'/ p2

# 运算符是一种语法糖，1 / 2  ==> instance.__xxx__(instance2)
# / 除法函数重新定义 /  Path类型中做除法符号(/)运算符重载
# + 'a' + 'b' => + 加号被运算符重载，用来描述拼接 concat('a', 'b')


class A:
    def __init__(self, x):
        self.x = x

    def __eq__(self, other):  # 同时解决 != __ne__
        return self.x == other.x

    def __gt__(self, other): # 同时解决 < __lt__
        return self.x > other.x

    def __ge__(self, other): # 同时解决 <= __le__
        return self.x >= other.x

    def __repr__(self):
        return "<A {}>".format(self.x)


# == 是比较容，同类型
# 不知道该怎么比较，同类型，转而比较内存地址是否相同
# 不通类型比较，返回False
print(1, A(5) == A(3))  # A(5).__eq__(A(3))  is
print(2, A(3) == A(3))
# print(3, 100 == A(3))
print(sorted([A(5), A(3), A(2)], reverse=True))
print(A(5) < A(3))
print(A(5) <= A(3))

In [ ]:
* 算术运算符重载

In [ ]:

class Student:
    def __init__(self, name, score):  # math
        self.name = name
        self.score = score

    def __sub__(self, other):
        return self.score - other.score
        # return self - other  # RecursionError

    def __isub__(self, other):  # i: inplace 就地修改
        # return Student(self.name, self.score - other.score)  # 重新构建一个Student实例
        # self.score -= other.score  # 在self对象上直接修改
        # return self
        return Student(self.name, self - other)

    def __repr__(self):
        return "<Student {}: {}>".format(self.name, self.score)


tom = Student('Tom', 90)
jerry = Student('Jerry', 85)
print(tom)

print(tom.score - jerry.score)
print(tom - jerry)  # tom.__sub__(jerry)
tom -= jerry  # tom = tom - jerry 如果有__isub__ 调用，如果没有转而调用tom.__sub__(jerry)
# tom类型发生了改变 Student -> int
print(tom)

In [27]:
a = [1]
b = [2]
print(a + b)  # 生成全新的对象
print(id(a), id(b))
a += b  # a = a + b  就地修改 地址不会改变 list += extend
print(a, b, sep="|")
print(id(a), id(b))

# list -> tuple 可以吗? 原理?

[1, 2]
1770901604168 1770900065352
[1, 2]|[2]
1770901604168 1770900065352


In [24]:
a = (1,)
b = (2,)
print(a + b)
print(id(a), id(b)) 

# 对元组来讲 tuplie += 不可以 元素不可变，元组为了实现这种+= 
# 转而调用 a+b 将结果赋值给a
a += b
print(a, b)
print(id(a), id(b))

(1, 2)
1770866890376 1770866968968
(1, 2) (2,)
1770912756168 1770866968968


In [31]:
a = 'a'
b = 'b'
print(a + b)
a += b
print(a, b)

ab
ab b


In [33]:
a = 100
b = 200
print(a + b)
a += b  # a = a + b 将 a+b的值 赋值给a
print(a, b)

300
300 200


* 上下文管理

- 上下文应用场景
  1. 增强功能
    - 在代码执行的前后增加代码，以增强其功能。类似装饰器的功能
    
  2. 资源管理
    - 打开了资源需要关闭，例如文件对象、网络连接、数据库连接等
  
  3. 权限验证
    - 在打开代码之前，做权限的验证，在\__enter__中处理
    

In [ ]:

# with open('test.ini') as f:  # 当时：f = open('test')
#     print(f)

# with 作用文件对象，文件要求必须支持上下文技术


# f = open('test.ini')
# with f as f1:  # f1 = f.__enter__() ==> f1 = f
#     print(1, f == f1)
#     print(2, f is f1)  # 离开with语句块的时候，调用文件对象.close()
# 文件对象.__exit__() 当作 会调用 文件对象.close()
# 不管with语句块中是否有异常发生，文件对象.__exit__()都一定执行


class A:
    def __init__(self):
        print('1 init +++++')

    def __enter__(self):  # self跟实例有关
        print('2 enter +++++')    # 打开资源
        return self
        # return 'abc'
        # with作用其后对象，调用__enter__魔术方法
        # 返回值可以作为as子句的标识符
        # 不是所有__enter__方法 都会返回自身对象

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('7 exit +++++')    # 释放资源
        print(exc_type, exc_val)
        # return [1]
        # 不管with块中是否有异常，都会执行
        # 如果返回值等价为True 压制异常
        # 等价False，异常继续向外抛出
        if isinstance(exc_val, ArithmeticError):
            return True
        # 可以压制所有异常，也可以压制指定异常


# 对一个对象来讲 当你进去的时候要干什么 __enter__
# 当你离开的时候 要干什么 __exit__
a1 = A()  # 对象
with a1 as a2:  # a2 = a1.__enter__()
    # 如果__enter__ return self a2 = a1
    print(a1, a2)
    print('3 line 1 +++++')
    print(4, a1 == a2)#, type(a1), type(a2))
    # 1/0
    raise Exception
    print(5, a1 is a2)
    print('6 line last +++++')
# 离开with语句块 __exit__

print('=' * 30)

# 需求：一开始就要干什么， 最后我们还要干什么  => 适用上下文管理
# e.g: 一开始打开资源，最后需要释放资源

In [36]:
import time
from functools import wraps
from _datetime import datetime


class Timeit:
    def __enter__(self):
        self.start = datetime.now()

    def __exit__(self, exc_type, exc_val, exc_tb):
        end = datetime.now()
        print("took {}s".format((end - self.start).total_seconds()))


def logger(wrapped):
    @wraps(wrapped)
    def wrapper(*args, **kwargs):
        start = datetime.now()
        ret = wrapped(*args, **kwargs)
        delta = (datetime.now() - start).total_seconds()
        print("Function {} took {}s.".format(wrapped.__name__, delta))
        return ret
    return wrapper


def add(x, y):
    time.sleep(2)
    return x + y


with Timeit():
    print(add(4, 5))

9
took 2.000325s


* 生成器函数 和 yield from
  - 上下文另一种实现 生成器函数实现，源码中非常常见
  - 生成器是一种特殊的迭代器，惰性求值
  
  * 1. generator expression
    - (x for x in range(10))
  
  * 2. 生成器函数
    - 生成器函数: 函数中只要有yield关键字就是
    - yield作用：把值扔出去，但是函数卡在这一句上暂停了


In [41]:
def a():
    for i in range(5):
        yield i

result = a()  # 生成器函数调用返回一个生成器对象，并没有真正的执行
print(type(result), result)
print(next(result))  # 100
print(next(result))  
print('=' * 30)
for i in result:
    print(i)
# print(next(result)) 
# print(next(result)) 
# print(next(result))  # next 碰到yield就暂停 
# print(next(result))  # return 抛出StopIteration

<class 'generator'> <generator object a at 0x0000019C51ECF0C8>
0
1
2
3
4


In [ ]:
import time
from contextlib import contextmanager


# 该装饰器作用的函数 必须是生成器函数
# 且只能yield一下 ***
@contextmanager
def a():
    print('之前增强 __enter__')
    # for i in range(5):
    #     yield i    # 用户态暂停，程序员写代码控制什么时候暂停
    # yield from range(5)   # 5下
    # 3.5之后 多使用yield from 语法糖
    yield 100
    print('之后增强 __exit__')


with a() as f:  # yield 值给了as之后的标识符
    print('object', f)    # 暂停在yeild 继续执行with语句块语句
    time.sleep(5)
    print('-' * 30)
    # 当with语句块执行完，执行yeild之后的语句

# result = a()
# # r2 = (i for i in range(10))
# print(type(result), result)
# print(next(result))
# print(next(result))
# print('=' * 30)
# for i in result:
#     print(i)
#
# for i in result:
#     print(i)

In [ ]:
** 生成器函数版本的上下文管理

In [ ]:
import time
from contextlib import contextmanager


# 该装饰器作用的函数 必须是生成器函数
# 且只能yield一下
@contextmanager
def a():
    # for i in range(5):
    #     yield i
    # yield from range(5)   # 5下
    # 3.5之后 多使用yield from 语法糖
    print('之前增强 __enter__')
    try:
        yield 100
    finally:
        print('之后增强 __exit__')


with a() as f:  # yield 值给了as之后的标识符
    print('object', f)    # 暂停在yeild 继续执行with语句块语句
    time.sleep(2)
    1/0
    print('-' * 30)
    time.sleep(2)
    # 当with语句块执行完，执行yeild之后的语句

In [ ]:
# 计算函数执行时间的装饰器的另一个写法
# 利用yield生成器函数计算时间

In [ ]:
import time
from datetime import datetime
from contextlib import contextmanager


@contextmanager
def a():
    print('之前增强 __enter__')
    start = datetime.now()
    try:
        yield 100
    finally:
        print('之后增强 __exit__')
        delta = (datetime.now() - start).total_seconds()
        print('took {}s'.format(delta))


def add(x, y):
    time.sleep(2)
    return x + y


with a() as f:
    print(add(4, 5))

* 计时器
* 监控哪些接口，执行时长过大

* 监控接口(函数)执行时间
  * 装饰器
  * 上下文管理
      * 类实现(__enter__ __exit__)
      * yield一下的生成器函数实现

In [ ]:
import time
from datetime import datetime
from contextlib import contextmanager


@contextmanager
def a(fn):
    print('之前增强 __enter__')
    start = datetime.now()
    try:
        yield fn
    finally:
        print('之后增强 __exit__')
        delta = (datetime.now() - start).total_seconds()
        print('{} took {}s'.format(fn.__name__, delta))


# py文件是一个模块 加载一次 sys.modules保证
# 顶级代码中的Timeit类对象 产生一次，类属性也就是一个
# 类对象存在，类属性也就构建好了
class Timeit():
    # start = datetime.now() 类属性，类定义时创建，N个实例化时固定
    def __init__(self, fn):
        self.__fn = fn

    def __enter__(self):
        self.start = datetime.now()
        return self.__fn

    @property
    def fn(self):
        return self.__fn

    def __exit__(self, exc_type, exc_val, exc_tb):
        delta = (datetime.now() - self.start).total_seconds()
        print('Function {} took {}s'.format(self.__fn.__name__, delta))


# add函数对象也就产生一次
def add(x, y):
    time.sleep(2)
    return x + y

# 上下文管理(class,yield) 和装饰器区别
# 上下文管理：可以统计with语句块里面所有语句执行时间
# 装饰器：仅装饰@下面的函数标识符
with Timeit(add) as f:   
    print(f(5, 5))
    print(f(2, 5))
    print(f(0, 5))


# with a(add) as f:    # f = add
#     print(f)
#     print(f(5, 6))

In [ ]:
** 模块化
  * 项目根目录下

* utils（目录） 包，本质依然是模块类型，文件夹(文件夹中有一个特殊文件__init__.py)或.py文件
* test（目录）， 里面没有__init__.py，它本质上也是包，(没有__init__.py)在某些时候会报错

* 包、目录、.py文件 本质上都是 模块 类型

* __init__.py 由于包本质上是目录，但是包又是标识符，包的直接资源怎么描述 => 包的直接资源就不得已采用一种特殊的方式，为包提供一个特殊的__init__.py

* 导入包，就是为当前编程环境提供一个标识符给程序员使用
* 导入包，就是加载包到内存，该包的直接资源__init__.py源代码被执行

* import后面 只能导入模块类型 **（包:文件夹 和 模块:.py文件）**


- utils    # 包
    - __init__.py
    - net    # 子包
        - __init__.py
    - web.py    # 子模块 导入 utils不加载子包、子模块(最小化原则)
- test   # 纯目录

```python
import test
import utils      # 只导入utils包，能用的标识符是utils。最小原则
import utils.net  # 导入utils和net包，但是能用的**标识符**是utils
```


* from utils import web   # import之后：模块类型(包 or 模块.py文件)、类、函数、变量，总结为标识符
  - utils 先导入，然后web子模块，能用的标识符是import后的标识符
  
* from utils.web import WEB
  - 导入utils模块，然后导入web模块，WEB就可以用了，标识符WEB
  
* from utils.net import socket
  - 导入utils模块，然后导入web模块，import子模块socket

* from 后必须是模块文件，因为模块是目录、py文件

* from utils.net.socket import connect, C  # 按需导入，推荐
* from utils.net.socket import *    # 容易造成变量污染， 不推荐。配合提供__all__ = {}
  - * 对于模块的_ __ 下划线 双下划线开头的 没法导入
  - 但是有了 __all__ = [], 如果使用了__all__ 对于*来说 就只能乖乖的遵照它了，解决from xxx import * 带来的大量变量污染
  
* from utils.net.socket import _E as E

* from ..web import WEB
  - 带点的表示模块的相对层级，这种相对导入只能用在包内
  - 相对导入的层次 就是目录的层级计算
  - 写了相对不能直接右键运行（不能作为主模块运行）
  - 只有 from 方式导入 支持相对

* 导入资源，资源指得是 模块、变量、类、函数等
* 项目根目录下的包， 绝对导入
* 相对导入，用在包内

**包名和模块名 必须小写；函数名一个小写；类名大驼峰；多个单词，建议使用_连接**


__all__ = []  # 阅读源码时，如果看到了它，就知道最重要的标识符时哪些


In [43]:
import os  # 父包
import os.path  # 建议这么写；
# 模块只加载一次，不会重复导入 记录在 sys.modules字典中，cache去重

print(os.path.exists(r'D:\xxx'))
print(os.path.exists(r'D:\JX3'))

# os.path 子包 子模块
# 上面能用的原因：因为python解释器必须加载的包 sys\os\io\built-in\os.path等

False
True


* Python中全局变量是什么？ 模块的的**顶级代码**中定义的标识符
* Python中全局变量有边界，模块。受模块的制约，不可以跨模块（跨模块使用 需要导入）

* 第一次加载 执行模块的顶级代码
* 导致之后 相关变量属于导入后模块的名词空间 globals()可以看到

* globals() 打印全局变量；权限变量受模块限制 不能跨模块
* locals() 打印局部变量；跟所处位置(作用域)有关

# 反射

* 运行时：runtime，区别于编译时，指得是程序被加载到内存中执行得时候
* 反射：reflection，指得是运行时获取类型定义信息

一个对象能够在运行时，像照镜子一样，反射出其类型信息。
简单说，在Python中，能够通过一个对象，找出其type、class、attribute或method的能力，称为反射或者自省

具有反射能力的函数有:
  - type()
  - isinstance()
  - callable()
  - dir()
  - getattr()等
  
内建函数：
- getarre(object, name[, default])  通过name返回object的属性值。当属性不存在，将使用default返回，如果没有default，则抛出AttributeError. name必须为**字符串**

- setattr(object, name, value)  object的属性存在，则覆盖，不存在，新增

- hasattr(object, name) 判断对象是否有这个名字的属性，name必须为**字符串**

In [ ]:
from functools import update_wrapper

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


p1 = Point(4, 5)
print(p1.x, p1.y)
print('-' * 30)

# getaddr setattr hasattr
print(1, getattr(p1, 'x'))  # name为 字符串
print(2, getattr(p1, 'y'))  # 等价== p1.y，都会用到 属性的搜索方式
print(3, p1.__dict__['y'])
print(4, getattr(p1, 'z', '10234'))  # AttributeError 实在找不到属性
# p1.z 无法使用缺省值
print(p1)
# setattr(p1, 'z', 100)
setattr(Point, 'z', 200)    # Point.z = 200 动态为类添加属性
print(p1.z)
# setattr(Point, '__str__', lambda self: "<Point {}: {} ++ >".format(self.x, self.y))
print(hasattr(Point, '__str__'))
# print(Point.__str__, '########')
# if not hasattr(Point, '__str__'):
Point.__str__ = lambda self: "<Point {}: {} &&>".format(self.x, self.y)
print(p1)    # print  p1.__str__  实例调用：绑定注入

In [ ]:
__getattr__

In [44]:
d = {'xyz': 'ttt', 'z': '12234'}


class Point:
    z = 1

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getattr__(self, item):
        print('getaddr ***', item, type(item))
        # 找不到的属性本身就不是它的，它转而调用别人的
        return d.get(item)


# __getattr__ __setattr__ __delattr__ 魔术方法
p1 = Point(10, 11)
print(p1.x)    # no __getattr__
print(p1.__dict__)
print(getattr(p1, 'x'))
print(p1.z)
print(p1.xyz)
# 如果 通过实例属性的搜索规则，找不到属性，按道理应该抛出AttributeError
# 但是 有了__getattr__，则调用该魔术方法

10
{'x': 10, 'y': 11}
10
1
getaddr *** xyz <class 'str'>
ttt


In [ ]:
__setattr__

In [45]:

# d = {}


class Point:
    z = 1

    def __init__(self, x, y):
        self.x = x
        self.y = y

    # def __getattr__(self, item):
    #     print('getaddr ***', item)
    #     return d.get(item)

    # 实例属性赋值时调用__setattr__ key='x' value=10
    def __setattr__(self, key, value):
        print('setattr!!!', key, value)
        # setattr(self, key, value)
        # 递归错误 => self.key = value 又调用 __setattr__

        # super(Point, self).__setattr__(key, value)
        # setattr(d, key, value)  # d.x = 10
        # d[key] = value
        self.__dict__[key] = value


# __getattr__ __setattr__ __delattr__ 魔术方法
p1 = Point(10, 11)
print(p1.x)
print(p1.__dict__)

setattr!!! x 10
setattr!!! y 11
10
{'x': 10, 'y': 11}


In [ ]:
__delattr__

In [ ]:

# d = {}


class Point:
    z = 1

    def __init__(self, x, y):
        self.x = x
        self.y = y

    # def __getattr__(self, item):
    #     print('getaddr ***', item)
    #     return d.get(item)

    # 实例属性赋值时调用__setattr__ key='x' value=10
    # def __setattr__(self, key, value):
    #     print('setattr!!!', key, value)
    #     # setattr(self, key, value)
    #     # 递归错误 => self.key = value 又调用 __setattr__
    #
    #     # super(Point, self).__setattr__(key, value)
    #     # setattr(d, key, value)  # d.x = 10
    #     # d[key] = value
    #     self.__dict__[key] = value

    def __delattr__(self, item):  # 通过实例删除一个属性
        print('del =', item)


# __getattr__ __setattr__ __delattr__ 魔术方法
p1 = Point(10, 11)
print(p1.x)
# print(p1.__dict__)
del p1.x
del p1.y
del p1.z

* 总结

  * __setattr__:  self.x = x（初始化赋值）或者 使用setattr()函数 该实例会触发该方法
  
  * __getattr__: 通过实例访问属性，如果找不到该属性(实例属性搜索规则)，会抛AttributeError的时候，会触发该方法，注意**返回值**(返回该值)
  
  * __delattr__: 删除属性时触发
  

In [ ]:
__getattribute__

In [ ]:
# d = {}


class Point:
    z = 1

    def __init__(self, x, y):
        self.x = x  # __setattr__
        self.y = y

    # 通过该类的实例访问属性，第一站都会使用这个方法
    def __getattribute__(self, item):
        print('attribute ***')
        # if item == '__dict__':
        #     return super().__getattribute__(item)
        # return object.__getattribute__(self, item)
        raise AttributeError

    def __getattr__(self, item):
        print('getaddr ***', item)
        return
    #
    # # 实例属性赋值时调用__setattr__ key='x' value=10
    def __setattr__(self, key, value):
        print('setattr!!!', key, value)
        # setattr(self, key, value)
        # 递归错误 => self.key = value 又调用 __setattr__

        # super(Point, self).__setattr__(key, value)
        super().__setattr__(key, value)
        # setattr(d, key, value)  # d.x = 10
        # d[key] = value
        # self.__dict__[key] = value

    # def __delattr__(self, item):  # 通过实例删除一个属性
    #     print('del =', item)


# __getattr__ __setattr__ __delattr__ 魔术方法
# __getattribute__
p1 = Point(10, 11)
print(p1.x)
print(p1.y)
print(p1.__dict__)
print(p1.z)
# print(p1.__dict__)
# del p1.x
# del p1.y
# del p1.z